# Data Collection and Cleaning

For this project we decided to investigate what factors have made mutual funds successful the last five years. In order to do this, we needed data on lots of funds. Yahoo finance (https://finance.yahoo.com/) has this type of data available free of charge but unfortunately, there is no provided API, so we decided to scrape the data. 

__NOTE:__ The code in this notebook takes a long time to run so we will not run this notebook before submission - we uploaded the data to the /share folder once we had it.

__NOTE:__ We are gathering data about funds but have used the terms stocks and funds interchangeably in variable names and folders.

In [1]:
# IMPORTS
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from os import listdir
%matplotlib inline

Every mutual fund has a unique symbol in addition to the fund name. For example, the fund Vanguard Institutional Index I has the symbol VINIX. In order to scrape data from funds, we needed the fund symbols because the URL:s for each fund included the fund symbol. We began by scraping 1488 fund symbols.

In [2]:
url = "https://finance.yahoo.com/screener/predefined/top_mutual_funds?count=1488"
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")
class_name = "data-col0 Ta(start) Pend(10px)"
links = soup.find_all("td", {"class": class_name})
symbols = []
for i in np.arange(0, len(links)): # len(links)
    link = links[i].find("a")
    symbols.append(link.get("data-symbol"))
    
symbols = pd.DataFrame({
        "symbol": symbols
    })
symbols.to_csv("/share/joe/fund_symbols.csv", index = False)

We uploaded the fund_symbols.csv-file to the /share-folder (so they are available with the command __pd.read_csv('/share/joeolof/fund_symbols.csv')__) and could now begin scraping data on individual funds. 
 d
We wanted historical price data (available from https://finance.yahoo.com/quote/VINIX/history?p=VINIX where VINIX is the fund symbol) and fund information (available from https://finance.yahoo.com/quote/VINIX/holdings?p=VINIX). Because the data is available from two different URL:s, we did this scraping in two separate steps using URL specific parameters for the BeautifulSoup objects.


### Stock Price Scraping

In [3]:
def getRowData(row):
    """Scrape date and closing price for a row (in our case, a month) from Yahoo Finance historical price data.
    Args: 
        row (BeautifulSoup object): A row from the "table.findAll("tr")"-command
    Returns: 
        dict: A dictionary containing the date and closing price scraped from the passed row.
    """
    cols = row.findAll('td')
    date = cols[0].get_text()
    
    if cols[1]['class'][0] == "Ta(c)":
        dividend = cols[1].get_text().strip()
        return {'dividend': dividend, 'date': date}

    close = float(cols[5].get_text().replace(',',''))
    return {'date': date, 'close':close}

In [4]:
def getFundData(symbol):
    """Scrape historical price data for a fund.
    Args:
        symbol (str): The fund symbol.
    Returns:
        DataFrame: A DataFrame containing historical price data for the fund.
    """
    url = "https://finance.yahoo.com/quote/%s/history?period1=1315551600&period2=1496473200&interval=1mo&filter=history&frequency=1mo" %symbol
    req = requests.get(url)
    bsobj = BeautifulSoup(req.text, "html.parser")
    table = bsobj.find('table', {'data-test':'historical-prices'}).find('tbody')
    rows = table.findAll('tr')
    data = pd.DataFrame([getRowData(row) for row in rows])
    return data

In [5]:
def saveFund(symbol):
    """Scrape and save historical price data for a fund.
    Args:
        symbol (str): The fund symbol.
    """
    data = getFundData(symbol)
    data.to_csv('/share/joe/stock_prices/%s.csv' %symbol, index=False)

We gathered the historical price data for each month since October 1, 2011. Since sometimes the code would encounter errors, we saved each fund's scraped data to a unique csv-file.

__NOTE:__ We commented out the cell below because it takes a long time to run

In [6]:
# # Scrape and save historical prices for all fund symbols in fund_symbols.csv
# symbols = pd.read_csv('/share/joeolof/fund_symbols.csv')
# for ix, symbol in enumerate(symbols['symbol']):
#     try:
#         print(ix,)
#         saveFund(symbol)
#     except:
#         print("\n\n\n\n\n\n\nERROR:\n\n\n\n\n\n\n\n", symbol)

Once we had historical price data on all of the funds, we uploaded all the files to /share/joe/stock_prices/ and combined them all into a single DataFrame.

In [7]:
symbols = pd.read_csv("/share/joeolof/fund_symbols.csv")
symbols = symbols.values[:, 0]
stockPricePath = "/share/joe/stock_prices/"
stockPrices = []
for symbol in symbols:
    df = pd.read_csv(stockPricePath + symbol + ".csv")
    df = df[['close', 'date']].dropna()
    df.index = pd.DatetimeIndex(df['date'])
    df.columns = [symbol, 'date']
    df = df[[symbol]]
    stockPrices.append(df)

In [8]:
stockPricesDf = pd.concat(stockPrices, axis = 1)
stockPricesDf.head()

BIPIX  BIPSX  UOPIX  UOPSX  RYVYX  RYCCX   DXQLX  FBIOX  FBTTX  \
date                                                                         
2011-10-01  12.01  10.41  28.84  25.50  22.10  19.20   85.45  62.21   7.54   
2011-11-01  11.47   9.92  27.12  23.96  20.77  18.03   81.00  63.50   7.69   
2011-12-01  12.19  10.53  26.64  23.52  20.36  17.67   79.67  65.50   7.94   
2012-01-01  14.45  12.48  31.19  27.52  23.87  20.69   92.84  73.85   8.93   
2012-02-01  14.41  12.44  35.28  31.09  26.98  23.38  104.67  74.52   9.03   

            FBTAX  ...    SAOAX  BPMIX  BPEAX  BPECX  KSRBX   VGELX  VGENX  \
date               ...                                                       
2011-10-01   7.80  ...    11.48    NaN    NaN    NaN  21.00   94.59  50.59   
2011-11-01   7.95  ...    11.60    NaN    NaN    NaN  20.21   95.58  51.12   
2011-12-01   8.20  ...    11.56    NaN    NaN    NaN  21.21   87.51  46.84   
2012-01-01   9.24  ...    12.20    NaN    NaN    NaN  23.07   96.54  51.63   
2012-02-01   9.34  ...    12.75    NaN    NaN    NaN  23.19  101.53  54.30   

            FSENX  FANIX  ENPSX  
date                             
2011-10-01  40.04  32.57  34.67  
2011-11-01  39.90  32.44  35.42  
2011-12-01  38.29  31.15  34.59  
2012-01-01  40.68  33.09  35.47  
2012-02-01  43.11  35.06  38.75  

[5 rows x 1488 columns]

In [9]:
# Save the historical prices to a csv file
stockPricesDf.to_csv('/share/joe/stockPrices.csv')

### Stock Information Scraping

In [10]:
def getAssetMakeup(obj):
    """Get the percentage of an individual asset
    Args:
        obj (BeautifulSoup object): a beautifulsoup row for a mutual fund's asset percentage of one kind of asset
    Returns:
        A tuple of the form (asset type, percentage of assets for this fund, percentage of assets for similar funds)
    """
    try1 = obj.find('span', {'class': 'Fl(end)'})
    if try1 is not None:
        return (obj.find('span', {'class':'Mend(5px) Whs(nw)'}).get_text(), try1.get_text())
    else:
        thisfund = obj.find('span', {'class': 'W(20%) D(b) Fl(start) Ta(e)'})
        avg = obj.find('span', {'class': 'W(30%) D(b) Fl(start) Ta(e)'})
        if avg is not None:
            return (obj.find('span', {'class':'Mend(5px) Whs(nw)'}).get_text(), thisfund.get_text(), avg.get_text())
        else:
            return (obj.find('span', {'class':'Mend(5px) Whs(nw)'}).get_text(), thisfund.get_text())

In [11]:
def getAssetMakeupFromSymbol(stockSymbol):
    """Get the asset percentages for a mutual fund based on its symbol
    Args:
        stockSymbol (str): a string representing a mutual fund
    Returns:
        a dataframe with the asset percentages of a mutual fund
    """
    url = "https://finance.yahoo.com/quote/%s/holdings?p=%s" %(stockSymbol, stockSymbol)
    req = requests.get(url)
    bsobj = BeautifulSoup(req.text, "html.parser")
    rows = bsobj.findAll('div', {'class':'Mb(25px)'})
    data = [getAssetMakeup(obj)
    for row in rows
    for obj in row.findAll('div', {'class': "Bdbw(1px) Bdbc($screenerBorderGray) Bdbs(s) H(25px) Pt(10px)"})]
    df = pd.DataFrame(data, columns=['item', 'thisFund', 'categoryAvg'])
    return df

In [12]:
def saveStock(symbol):
    """Scrape and save asset percentages for a mutual fund based on its symbol
    Args:
        symbol (str): a string representing a mutual fund
    """
    data = getAssetMakeupFromSymbol(symbol)
    data.to_csv("/share/joe/stock_info/" + symbol + ".csv", index=False)

In [13]:
# symbols = pd.read_csv('/share/joeolof/fund_symbols.csv')
# for ix, symbol in enumerate(symbols['symbol']):
#     try:
#         saveStock(symbol)
#     except:
#         print(ix, symbol)

In [14]:
getAssetMakeupFromSymbol("FSSPX")

item  thisFund categoryAvg
0                     Cash     2.26%        None
1                   Stocks    97.67%        None
2                    Bonds     0.07%        None
3                   Others     0.00%        None
4                Preferred     0.00%        None
5              Convertable     0.00%        None
6          Basic Materials     6.07%        None
7        Consumer Cyclical    11.86%        None
8       Financial Services    18.67%        None
9               Realestate     8.72%        None
10      Consumer Defensive     3.59%        None
11              Healthcare    11.76%        None
12               Utilities     3.56%        None
13  Communication Services     1.19%        None
14                  Energy     3.16%        None
15             Industrials    14.48%        None
16              Technology    16.92%        None
17          Price/Earnings     20.41       19.37
18              Price/Book      2.02        2.26
19             Price/Sales      1.19        1.13
20          Price/Cashflow      7.94        9.28
21       Median Market Cap  1,711.56    2,998.49
22  3 Year Earnings Growth      10.1       14.11
23                Maturity       N/A        2.23
24                Duration       N/A         1.1
25          Credit Quality       N/A         N/A
26            US Goverment     0.00%        None
27                     AAA     0.00%        None
28                      AA     0.00%        None
29                       A     0.00%        None
30                     BBB     0.00%        None
31                      BB     0.00%        None
32                       B     0.00%        None
33                 Below B     0.00%        None
34                  Others     0.00%        None

Now we gather all the asset percentage data we collected and combine it into a single dataframe

In [15]:
files = listdir("/share/joe/stock_info")
dfs = []
for file in files:
    df = pd.read_csv("/share/joe/stock_info/" + file)
    newdf = df.transpose()
    cols = list(df['item'])
    if len(cols) == 0:
        # the dataframe is empty, so ignore it
        continue
    cols[3] = 'otherAssets'
    newdf.columns = cols
    newdf = newdf.drop(['item', 'categoryAvg'])
    newdf.index = [file.split('.')[0]]
    
    dfs.append(newdf)

In [16]:
assetData = pd.concat(dfs)

for column in assetData.columns:
    assetData[column] = assetData[column].map(lambda x: float(str(x).replace('%', '').replace(',', '')))

assetData.head()

Cash  Stocks  Bonds  otherAssets  Preferred  Convertable  \
LZEMX  3.00   96.97    0.0         0.03       0.00          0.0   
FIDGX  0.57   99.43    0.0         0.00       0.00          0.0   
SVFDX  3.92   96.08    0.0         0.00       0.00          0.0   
GICPX  0.33   99.67    0.0         0.00       0.00          0.0   
JDMNX  8.00   90.87    0.0         0.88       0.25          0.0   

       Basic Materials  Consumer Cyclical  Financial Services  Realestate  \
LZEMX             4.40              10.03               27.69        0.00   
FIDGX             6.41              15.45                8.63        2.21   
SVFDX             0.00              31.69               35.76        0.00   
GICPX             2.61              25.71                7.14        0.00   
JDMNX             0.27              12.29                8.82        3.87   

        ...    Credit Quality  US Goverment  AAA   AA    A  BBB   BB    B  \
LZEMX   ...               NaN           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
FIDGX   ...               NaN           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
SVFDX   ...               NaN           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
GICPX   ...               NaN           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
JDMNX   ...               NaN           0.0  0.0  0.0  0.0  0.0  0.0  0.0   

       Below B  Others  
LZEMX      0.0     0.0  
FIDGX      0.0     0.0  
SVFDX      0.0     0.0  
GICPX      0.0     0.0  
JDMNX      0.0     0.0  

[5 rows x 35 columns]

Credit quality is always nan, so we drop that row

In [17]:
assetData = assetData.drop('Credit Quality', axis = 1)

In [18]:
# Right now our index is fund symbols, lets make fund symbols their own column
assetData = assetData.reset_index()
assetData['symbol'] = assetData['index']
assetData = assetData.drop('index', axis = 1)

In [19]:
assetData.head()

Cash  Stocks  Bonds  otherAssets  Preferred  Convertable  Basic Materials  \
0  3.00   96.97    0.0         0.03       0.00          0.0             4.40   
1  0.57   99.43    0.0         0.00       0.00          0.0             6.41   
2  3.92   96.08    0.0         0.00       0.00          0.0             0.00   
3  0.33   99.67    0.0         0.00       0.00          0.0             2.61   
4  8.00   90.87    0.0         0.88       0.25          0.0             0.27   

   Consumer Cyclical  Financial Services  Realestate   ...    US Goverment  \
0              10.03               27.69        0.00   ...             0.0   
1              15.45                8.63        2.21   ...             0.0   
2              31.69               35.76        0.00   ...             0.0   
3              25.71                7.14        0.00   ...             0.0   
4              12.29                8.82        3.87   ...             0.0   

   AAA   AA    A  BBB   BB    B  Below B  Others  symbol  
0  0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0   LZEMX  
1  0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0   FIDGX  
2  0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0   SVFDX  
3  0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0   GICPX  
4  0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0   JDMNX  

[5 rows x 35 columns]

In [20]:
# Save the stock information DataFrame to a csv-file
assetData.to_csv('/share/joe/cleanStockInfo.csv', index=False)

Now we'll gather some summary statistics on each mutual fund: turnover rate, expense ratio, net asset value, the date the fund started, the yield, and the three year beta of the fund

In [21]:
def getData(symbol):
    """Scrape summary statistics for a fund.
    Args:
        symbol (str): The fund symbol.
    Returns:
        A Dictionary with fund information.
    """
    url = "https://finance.yahoo.com/quote/%s/" % symbol
    req = requests.get(url)
    bsobj = BeautifulSoup(req.text, 'html.parser')
    turnover = bsobj.find('td', {'data-test': "ANNUAL_HOLDINGS_TURNOVER-value"}).get_text()
    expRatio = bsobj.find('td', {'data-test': "EXPENSE_RATIO-value"}).get_text()
    NAV = bsobj.find('td', {'data-test': "NET_ASSETS-value"}).get_text()
    inception = bsobj.find('td', {'data-test': "FUND_INCEPTION_DATE-value"}).get_text()
    yieldVal = bsobj.find('td', {'data-test': "TD_YIELD-value"}).get_text()
    beta3y = bsobj.find('td', {'data-test': "BETA_3Y-value"}).get_text()
    return {'symbol': symbol,
            'turnover': turnover,
           'expRatio': expRatio,
           'netAssetValue': NAV,
           'inceptionDate': inception,
           'yield': yieldVal,
           'beta3y': beta3y}


In [22]:
# # Scrape and save fund information for all funds whose symbols are in fund_symbols.csv
# stockInfo = []
# symbols = pd.read_csv('/share/joeolof/fund_symbols.csv')
# for ix, symbol in enumerate(symbols['symbol']):
#     try:
#         stockInfo.append(getData(symbol))
#     except:
#         print("\nfailure")
#         print(ix, symbol)
#         print('\n\n')
#  summaryData = pd.DataFrame(stockInfo)
#  summaryData.to_csv('/share/joe/moreStockInfo.csv', index=False)

In [23]:
info = pd.read_csv('/share/joe/moreStockInfo.csv') # Read in stock information data

def replacePercent(string):
    '''Remove percentage and comma characters
    Args:
        string (str): A string with percentages
    Returns:
        Passed string with percentage and comma characters removed
    '''
    if type(string) == str:
        return float(string.replace('%', '').replace(',', ''))
    else:
        return string

# Clean expRatio, turnover, and yield columns
for symbol in ['expRatio', 'turnover', 'yield']:
    info[symbol] = info[symbol].map(replacePercent)
    
def convert_MB(netAssetValues):
    '''Convert net asset values to billions
    Args:
        netAssetValues (series): A series with net asset values as strings
    Returns:
        Series with net asset values as floats in billions
    '''
    ret = []
    for netAssetValue in netAssetValues:
        if "M" in str(netAssetValue):
            ret.append(float(netAssetValue[:-1]) / 1000)
        elif "B" in str(netAssetValue):
            ret.append(float(netAssetValue[:-1]))
        elif "k" in str(netAssetValue):
            ret.append(float(netAssetValue[:-1]) / (10 ** 6))
        else:
            ret.append(np.NaN)
    return np.array(ret)

# Create column == 1 if net asset value is > 1 billion, 0 otherwise
info['over1B'] = info['netAssetValue'].str.contains('B')
# Get net asset values as floats in billion dollars
info['netAssetValue'] = convert_MB(info['netAssetValue'])
# Keep only inception year; remove day and month
info['inceptionDate'] = info['inceptionDate'].map(lambda x: int(x[-4:]))

# Save fund summary data to a csv-file
info.to_csv('/share/joe/fundSummaryInfo.csv', index=False)

We'll add the percent that a mutual fund has changed since 5 years ago, then merge all of the data together and save it into a csv file for future use

In [24]:
# Limit dates to 5 years ago, limit to funds that have been around at least 5 years.
stock_prices = pd.read_csv("/share/joe/stockPrices.csv")
stock_prices = stock_prices.drop(range(0, 8)).dropna(axis = 1)
stock_prices.index = range(len(stock_prices["date"]))
 
# Get percentage increase
start_prices = stock_prices.iloc[0, 1:]
latest_prices = stock_prices.iloc[-1, 1:]
percents = pd.DataFrame((latest_prices / start_prices))

# merge all of the data we are going to use into one data frame
price_change = percents.reset_index()
price_change.columns = ['symbol', 'changeInPrice']
info = pd.read_csv('/share/joe/fundSummaryInfo.csv')
moreInfo = pd.read_csv('/share/joe/cleanStockInfo.csv')
finalData = info.merge(price_change, on='symbol')
finalData = finalData.merge(moreInfo, on='symbol')

In [25]:
finalData.to_csv('/share/joe/finalData.csv', index=False)